In [51]:
include("../load.jl")

pool1

In [57]:
include("../../data/speed_reducer.jl")

speed_reducer (generic function with 2 methods)

In [58]:
gm = sagemark_to_GlobalModel(3; lse=false)
gm = speed_reducer()
set_optimizer(gm, CPLEX_SILENT)
set_param(gm, :abstol, 1e-7)

# Actually trying to optimize...
find_bounds!(gm, all_bounds=true)
uniform_sample_and_eval!(gm)

surveysolve(gm)
println("Approximation accuracies: ", evaluate_accuracy(gm))

┌ Info: bbl2 was not KNN sampled since it has no infeasible samples.
└ @ Main.OptimalConstraintTree C:\Users\berko\Code\OptimalConstraintTree\src\sample.jl:154
┌ Info: bbl3 was not KNN sampled since it has no infeasible samples.
└ @ Main.OptimalConstraintTree C:\Users\berko\Code\OptimalConstraintTree\src\sample.jl:154
┌ Info: bbl5 was not KNN sampled since it has no infeasible samples.
└ @ Main.OptimalConstraintTree C:\Users\berko\Code\OptimalConstraintTree\src\sample.jl:154
┌ Info: bbl8 was not KNN sampled since it has no infeasible samples.
└ @ Main.OptimalConstraintTree C:\Users\berko\Code\OptimalConstraintTree\src\sample.jl:154
Training trees...                  100%|████████████████| Time: 0:00:00
Training trees...                  100%|████████████████| Time: 0:00:00
Training trees...                  100%|████████████████| Time: 0:00:00
Training trees...                  100%|████████████████| Time: 0:00:00
Training trees...                  100%|████████████████| Time: 0:00:00


Approximation accuracies: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


┌ Warning: Accuracy of BlackBoxClassifier bbl2 is tautological.
└ @ Main.OptimalConstraintTree C:\Users\berko\Code\OptimalConstraintTree\src\global_model.jl:350
┌ Warning: Accuracy of BlackBoxClassifier bbl3 is tautological.
└ @ Main.OptimalConstraintTree C:\Users\berko\Code\OptimalConstraintTree\src\global_model.jl:350
┌ Warning: Accuracy of BlackBoxClassifier bbl5 is tautological.
└ @ Main.OptimalConstraintTree C:\Users\berko\Code\OptimalConstraintTree\src\global_model.jl:350
┌ Warning: Accuracy of BlackBoxClassifier bbl8 is tautological.
└ @ Main.OptimalConstraintTree C:\Users\berko\Code\OptimalConstraintTree\src\global_model.jl:350


In [59]:
# Solving of model
set_param(gm, :ignore_accuracy, true)
# add_tree_constraints!(gm)
# optimize!(gm)
vals = solution(gm);
add_infeasibility_cuts!(gm)
optimize!(gm)
while abs(gm.cost[end] - gm.cost[end-1]) > get_param(gm, :abstol)
    add_infeasibility_cuts!(gm)
    optimize!(gm)
end
println("X values: ", vals)
println("Optimal X: ", vcat(exp.([5.01063529, 3.40119660, -0.48450710]), [-147-2/3]))

X values: 1×8 DataFrame
 Row │ x[1]     x[2]      x[3]     x[4]     x[5]     x[6]     x[7]     x[8]    
     │ Float64  Float64   Float64  Float64  Float64  Float64  Float64  Float64 
─────┼─────────────────────────────────────────────────────────────────────────
   1 │ 3.50191  0.700383     17.0      7.3  7.71572  3.34829  5.28702  2990.75
Optimal X: [149.9999993855616, 29.999976550144503, 0.6160007487168032, -147.66666666666666]


┌ Info: 1 infeasibility cuts added.
└ @ Main.OptimalConstraintTree C:\Users\berko\Code\OptimalConstraintTree\src\refine.jl:229
┌ Info: 0 infeasibility cuts added.
└ @ Main.OptimalConstraintTree C:\Users\berko\Code\OptimalConstraintTree\src\refine.jl:229


In [60]:
gm.bbls[1].learners[end]
gm.bbls[1].actuals
# gm.bbls[1].mi_constraints

3-element Array{Any,1}:
 2996.601100707525
 2997.0960141403775
 2997.0960141403775

In [61]:
IAI.show_in_browser(gm.bbls[2].learners[end])

"C:\\Users\\berko\\AppData\\Local\\Temp\\jl_E2BA.tmp.html"

In [ ]:
bbr = gm.bbls[1]
score = IAI.score(bbr.learners[end], bbr.X, bbr.Y)

In [ ]:
gm.model

In [63]:
# Printing numbers of variables and constraints
println("Variables: $(length(JuMP.all_variables(gm.model)))")
println("Constraints: $(sum(length(all_constraints(gm.model, type[1], type[2])) for type in JuMP.list_of_constraint_types(gm.model))
)")

Variables: 275
Constraints: 716


In [ ]:
# Solving via Ipopt